# Financial-IA — Latent Market Intelligence Demo (v4)

**End-to-end demo of the Strate IV PPO agent** operating in latent space learned by Fin-JEPA.

This notebook:
1. Installs dependencies and clones the repo
2. Downloads pre-trained checkpoints (PPO agent + VecNormalize + trajectory buffer)
3. Runs the agent on held-out evaluation episodes
4. Visualizes regime switching, position management, and PnL vs Buy & Hold

**No training required** — inference only (~30 seconds on CPU, ~5s on GPU).

---
> Architecture: Spherical VQ-VAE → Fin-JEPA (Mamba-2) → Stochastic Predictor → PPO Agent  
> Paper reference: LeCun (2022) *A Path Towards Autonomous Machine Intelligence* — JEPA framework

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q torch pytorch-lightning tslearn numpy pandas dacite pyyaml einops gymnasium stable-baselines3 matplotlib

In [ ]:
import os

# Clone the repo (skip if already cloned)
if not os.path.exists('World-IA-Finance'):
    !git clone https://github.com/ElMonstroDelBrest/World-IA-Finance.git

os.chdir('World-IA-Finance')
print('Working directory:', os.getcwd())

## 2. Download Pre-trained Checkpoints

In [ ]:
import urllib.request
import zipfile
from pathlib import Path

BASE_URL = "https://github.com/ElMonstroDelBrest/World-IA-Finance/releases/download/v4.0.0"

def download(url, dest):
    dest = Path(dest)
    if dest.exists():
        print(f'  {dest} already exists, skipping.')
        return
    dest.parent.mkdir(parents=True, exist_ok=True)
    print(f'  Downloading {dest.name}...')
    urllib.request.urlretrieve(url, dest)
    print(f'  Done ({dest.stat().st_size / 1e6:.1f} MB)')

# PPO model checkpoint (v4, trained with VecNormalize)
print('Downloading PPO model...')
download(
    f"{BASE_URL}/best_model.zip",
    "checkpoints/strate_iv/best_model.zip"
)

# VecNormalize stats (MUST match the model)
print('Downloading VecNormalize stats...')
download(
    f"{BASE_URL}/vecnormalize.pkl",
    "checkpoints/strate_iv/vecnormalize.pkl"
)

# Pre-computed trajectory buffer (JEPA latent representations)
print('Downloading trajectory buffer...')
download(
    f"{BASE_URL}/trajectory_buffer_v4.zip",
    "/tmp/trajectory_buffer_v4.zip"
)

# Extract trajectory buffer
buf_dir = Path('data/trajectory_buffer_v4')
if not buf_dir.exists() or not any(buf_dir.glob('*.pt')):
    print('Extracting trajectory buffer...')
    with zipfile.ZipFile('/tmp/trajectory_buffer_v4.zip', 'r') as z:
        z.extractall('.')
    print(f'  Extracted {len(list(buf_dir.glob("*.pt")))} episodes')
else:
    print(f'  Buffer already extracted: {len(list(buf_dir.glob("*.pt")))} episodes')

print('\nAll assets ready.')

## 3. Run Agent Demo

The PPO agent operates on **latent observations** — not raw prices. Each step, it receives:
- The JEPA context encoding of past market regimes
- A distribution of N=16 stochastic future trajectories (mean & std at current step)
- Close return statistics, volatility regime, macro trend
- Its current position and cumulative PnL

It outputs a continuous action in `[-1, 1]` (short → flat → long).

**v4**: Observations are normalized via VecNormalize (loaded from `vecnormalize.pkl`). The obs_dim is auto-detected from the trajectory buffer.

In [ ]:
import sys, os
sys.path.insert(0, '.')

import numpy as np
import matplotlib
matplotlib.use('Agg')
from pathlib import Path
from IPython.display import display, Image

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from src.strate_iv.config import load_config
from src.strate_iv.env import LatentCryptoEnv
from src.strate_iv.trajectory_buffer import TrajectoryBuffer
from scripts.demo_results import run_episode, plot_demo

# Load config and buffer
config = load_config('configs/strate_iv.yaml')
buffer = TrajectoryBuffer('data/trajectory_buffer_v4/')
_, eval_buffer = buffer.split(val_ratio=config.buffer.val_ratio)
print(f'Eval buffer: {len(eval_buffer)} episodes')

# Create env — obs_dim auto-detected from buffer
env = LatentCryptoEnv(buffer=eval_buffer, config=config.env)
obs_dim = env.observation_space.shape[0]
print(f'Auto-detected obs_dim: {obs_dim}')

# Load VecNormalize + PPO model
vecnorm_path = 'checkpoints/strate_iv/vecnormalize.pkl'
model_path = 'checkpoints/strate_iv/best_model.zip'

vec_env = DummyVecEnv([lambda: LatentCryptoEnv(buffer=eval_buffer, config=config.env)])
vec_env = VecNormalize.load(vecnorm_path, vec_env)
vec_env.training = False
vec_env.norm_reward = False

model = PPO.load(model_path, env=vec_env)
print(f'PPO model loaded — obs dim: {model.observation_space.shape[0]}')
print(f'VecNormalize loaded from {vecnorm_path}')
print(f'Window: {config.env.n_tgt} patches × {config.env.patch_len} candles = {config.env.n_tgt * config.env.patch_len}h')

In [ ]:
# Run episodes through the VecNormalize-wrapped env
Path('outputs/demo').mkdir(parents=True, exist_ok=True)
n_demos = 5
results = []

print(f'Running {n_demos} episodes...\n')
for i in range(n_demos):
    traj = run_episode(model, vec_env, seed=None, use_vec=True)
    out_path = f'outputs/demo/demo_v4_{i:02d}.png'
    plot_demo(traj, out_path)
    results.append((out_path, traj))
    print(f'  Episode {i+1}: actions = {traj["actions"].round(2).tolist()}')

## 4. Results

In [ ]:
from IPython.display import display, Image

for i, (out_path, traj) in enumerate(results):
    print(f'\n--- Episode {i+1} | actions = {traj["actions"].round(2).tolist()} ---')
    display(Image(filename=out_path, width=900))


## 5. Summary

### What you just saw

The agent **never sees raw prices** during inference. It operates entirely on latent representations produced by Fin-JEPA:

| Component | Role |
|---|---|
| **Spherical VQ-VAE** (Strate I) | Tokenizes OHLCV patches → discrete market regime tokens |
| **Fin-JEPA + Mamba-2** (Strate II) | Self-supervised temporal model over token sequences |
| **Stochastic Predictor** (Strate III) | Samples N=16 divergent future latent trajectories |
| **PPO Agent** (Strate IV) | Plans in latent space, outputs continuous position [-1, 1] |

### v4 improvements

- **VecNormalize**: Observations are normalized (JEPA latents have norms of ~50M). The `vecnormalize.pkl` is always saved alongside the model.
- **Auto-detected obs_dim**: No hardcoded fallback — obs structure is always derived from the trajectory buffer.
- **600 episodes**: Larger trajectory buffer for better coverage of market regimes.

### Why this matters

Classical approaches (LSTM, Transformer on raw prices) are forced to predict **every tick** — memorizing noise instead of learning structure.  
JEPA learns to predict **latent representations** of future states, ignoring unpredictable details.  
The agent then plans in this cleaner latent space, exhibiting genuine **regime switching** rather than curve-fitting.

---

**Repository:** https://github.com/ElMonstroDelBrest/World-IA-Finance  
**License:** AGPL-3.0